In [1]:
import cv2
import os

import paths
from process_dataset import common

In [2]:
# %pip install -U openmim

# !mim install "mmcv>=2.0.0rc1,<2.1.0"

# %cd $paths.mm_parent_dirpath
# # !git clone https://github.com/open-mmlab/mmdetection.git
# %cd mmdetection
# %pip install -e .
# %cd $paths.proj_path

# %cd $paths.mm_parent_dirpath
# # !git clone https://github.com/open-mmlab/mmengine
# %cd mmengine
# %pip install -e .
# %cd $paths.proj_path

In [3]:
# Epoch number (to choose a checkpoint)
e = 23

device = "cuda:0"
# device = "cpu"

score_thr = 0.6
# score_thr = 0

# Video input filename
input_filename = "day_hq.mp4"

output_name = input_filename.split(".")[0] + f"_e{e}_{score_thr}"

# Video output filename
output_filename = output_name + ".mp4"

# Step (every n-th frame will be taken, other ignored)
# step = 1
step = 30

input_fps = 30

# Frames output folder
frames_dirname = output_name

# Pick a specific .pth file
paths.last_checkpoint_filepath = os.path.join(paths.working_dirpath, f"epoch_{e}.pth")

# stop_at_frames = 0
stop_at_frames = 100

In [4]:
print(paths.model_config_filepath)
print(paths.last_checkpoint_filepath)

c:/Users/z004ktej/Desktop/d_projekty/bp/proj\..\mmdetection\configs\yolox\yolox_s_8xb8-300e_coco_custom.py
c:/Users/z004ktej/Desktop/d_projekty/bp/proj\working_dir_yolox_4_bad\epoch_23.pth


Error `UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.`
can be ignored. Just now, even after receiving this error, images are annotated correctly with my classes.

In [6]:
import mmcv
from mmdet.apis import init_detector, inference_detector

# MMDetection
# Check for v2.0:
# `video_demo.py ../../proj/day_hq.mp4 /Users/z004ktej/Desktop/d_projekty/bp/proj\..\mmdetection_old\configs\yolox\yolox_s_8x8_300e_coco_custom.py "/Users/z004ktej/Desktop/d_projekty/bp/proj\working_dir_yolox_3\epoch_23.pth" --show`
# Check for v3.0:
from mmdet.registry import VISUALIZERS

# MMYolo TODO
# Check:
# `python video_demo.py ../../proj/day_hq.mp4 "/Users/z004ktej/Desktop/d_projekty/bp/proj\..\mmyolo\configs\yolov8\yolov8_s_syncbn_fast_8xb16-500e_coco_custom.py" "/Users/z004ktej/Desktop/d_projekty/bp/proj\working_dir_yolox_3\epoch_23.pth" --show`
# from mmyolo.utils import register_all_modules
# from mmyolo.registry import VISUALIZERS
# register_all_modules()

model = init_detector(paths.model_config_filepath, paths.last_checkpoint_filepath, device=device)

model.CLASSES = list(common.classes_ids.keys()) # Needed for correct class names

model.cfg.test_dataloader.dataset.pipeline[0].type = "mmdet.LoadImageFromNDArray"

# print(model.cfg.pretty_text)

visualizer = VISUALIZERS.build(model.cfg.visualizer)
visualizer.dataset_meta = model.dataset_meta

# test_pipeline = mmcv.transforms.Compose(model.cfg.test_dataloader.dataset.pipeline)

# I wanted to use the original yolox config and just adjust the number of 
# classes and classes themselves here, but this strangely results in misdetections
# when inferencing... So, in this script, it is necessary to use a custom config
# where the number of classes is updated
# model.bbox_head.num_classes = len(common.classes)

video = mmcv.VideoReader(input_filename)

frames_dir = os.path.join(paths.working_dirpath, frames_dirname)
if not os.path.exists(frames_dir):
    os.mkdir(frames_dir)

counter = 0
try:
    for frame in video:
        print(counter // step, end="\r")

        if counter % step == 0:

            out_filepath = os.path.join(frames_dir, str(counter // step).zfill(6) + ".jpg")

            # in_filepath = os.path.join(os.path.dirname(out_filepath), "tmp.jpg")
            # cv2.imwrite(in_filepath, frame)
            # result = inference_detector(model, in_filepath)
            # result = inference_detector(model, frame, test_pipeline=test_pipeline)
            # result["img_path"] = tmp

            result = inference_detector(model, frame)
            print(result)

            visualizer.add_datasample(
                os.path.basename(out_filepath),
                frame,
                data_sample=result,
                draw_gt=False,
                show=False,
                wait_time=0,
                out_file=out_filepath,
                pred_score_thr=score_thr
            )
            # annotated = visualizer.get_image()
            # cv2.imwrite(out_filepath, annotated)

            # Without mmyolo, this worked:
            # if score_thr == 0:
            #     model.show_result(frame, result, out_file=out_filepath)
            # else:
            #     model.show_result(frame, result, score_thr=score_thr, out_file=out_filepath)

        counter += 1
        if counter == stop_at_frames * step: # Doesn't stop if stop_at_frames == 0
            raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Stop requested")

KeyError: "class `YOLOX` in mmdet/models/detectors/yolox.py: 'DetDataPreprocessor is not in the model registry. Please check whether the value of `DetDataPreprocessor` is correct or it was registered as expected. More details can be found at https://mmengine.readthedocs.io/en/latest/advanced_tutorials/config.html#import-the-custom-module'"

In [ ]:
fps = input_fps // step
mmcv.frames2video(frames_dir, os.path.join(paths.working_dirpath, output_filename), fps=fps)